# Growth accounting

In this notebook we conceptually replicates Table 5.1 from Aghion and Howitt.

We turn to the Penn World Table again. First we load the data and define our variables.

In [1]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:,.2f}'.format


# Download data from Penn World Table
url = "https://www.rug.nl/ggdc/docs/pwt100.xlsx"

data = pd.read_excel(url, sheet_name='Data')

data = data.assign(capital_intensity = data['rnna']/data['emp'])
data = data.assign(gdp_per_capita = data['rgdpna']/data['emp'])
index = pd.MultiIndex.from_arrays([
    data['country'], 
    data['year']], 
    names=['Country', 'year'])

data.index = index
data[['countrycode', 'country', 'currency_unit', 'year', 'rgdpe', 'rgdpo']]


countrycode   country   currency_unit  year     rgdpe     rgdpo
Country  year                                                                
Aruba    1950         ABW     Aruba  Aruban Guilder  1950       NaN       NaN
         1951         ABW     Aruba  Aruban Guilder  1951       NaN       NaN
         1952         ABW     Aruba  Aruban Guilder  1952       NaN       NaN
         1953         ABW     Aruba  Aruban Guilder  1953       NaN       NaN
         1954         ABW     Aruba  Aruban Guilder  1954       NaN       NaN
...                   ...       ...             ...   ...       ...       ...
Zimbabwe 2015         ZWE  Zimbabwe       US Dollar  2015 40,141.62 39,798.64
         2016         ZWE  Zimbabwe       US Dollar  2016 41,875.20 40,963.19
         2017         ZWE  Zimbabwe       US Dollar  2017 44,672.18 44,316.74
         2018         ZWE  Zimbabwe       US Dollar  2018 44,325.11 43,420.90
         2019         ZWE  Zimbabwe       US Dollar  2019 42,296.06 40,826.57

[12810 rows x 6 columns]

We then perform our analysis. First we construct GDP growth and capital deepening from our measures of GDP per capita and capital intensity. Then we assign a value to $\alpha$ and back out TFP growth. Finally, we compute the share of GDP growth which is attributed to TFP and capital.

In [2]:
start_year = 1960
end_year = 2015
years = end_year - start_year

gdp_growth = np.log(data.xs(end_year, level='year')['gdp_per_capita']) - np.log(data.xs(start_year, level='year')['gdp_per_capita'])
capital_deepening = np.log(data.xs(end_year, level='year')['capital_intensity']) - np.log(data.xs(start_year, level='year')['capital_intensity'])

alpha = 0.3
tfp_growth = gdp_growth - alpha*capital_deepening

tfp_share = tfp_growth/gdp_growth
capital_share = 1- tfp_share

/Users/kalle/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


After having done that, we simply produce our table for the OECD countries.

In [3]:
#Create a new table
growth_accounting_table = pd.DataFrame([100*gdp_growth/years, 100*tfp_growth/years, 
                                        100*alpha*capital_deepening/years, tfp_share, capital_share]).transpose()
growth_accounting_table.columns = ['Growth Rate', 'TFP Growth', 
                                   'Capital Deepening', 'TFP Share', 'Capital Share']

#Only print for OECD countries:
oecd = ['Australia', 'Austria', 'Belgium', 
        'Canada', 'Denmark', 'Finland',
        'France', 'Germany', 'Greece', 
        'Iceland', 'Ireland', 'Italy', 
        'Japan', 'Netherlands', 'New Zealand', 
        'Norway', 'Portugal', 'Spain', 
        'Sweden', 'Switzerland', 'United Kingdom', 
        'United States']

growth_accounting_table.loc[oecd]

,Growth Rate,TFP Growth,Capital Deepening,TFP Share,Capital Share
Country,,,,,
Australia,1.52,1.17,0.36,0.77,0.23
Austria,2.33,1.48,0.84,0.64,0.36
Belgium,2.16,1.62,0.54,0.75,0.25
Canada,1.27,0.78,0.48,0.62,0.38
Denmark,1.84,1.17,0.67,0.63,0.37
Finland,2.59,1.68,0.91,0.65,0.35
France,2.25,1.57,0.68,0.70,0.30
Germany,2.15,1.36,0.78,0.64,0.36
Greece,2.60,1.70,0.90,0.65,0.35


And finally we display the average values as well...

In [4]:
averages = pd.DataFrame(growth_accounting_table.loc[oecd].mean()).transpose()
averages.index = ['Average']
averages

,Growth Rate,TFP Growth,Capital Deepening,TFP Share,Capital Share
Average,2.11,1.42,0.69,0.67,0.33
